<a href="https://colab.research.google.com/github/CJunbot/Dacon_Jeju_traffic/blob/master/Dacon_Jeju_traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!nvidia-smi

Wed Oct 12 00:22:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
!git clone --recursive https://github.com/Microsoft/LightGBM 
!cd LightGBM && mkdir build && cd build
!cmake -DUSE_GPU=1 ..
!make -j4
!pip uninstall lightgbm
!cd ../python-package/ && python setup.py install

Cloning into 'LightGBM'...
remote: Enumerating objects: 27695, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (197/197), done.
remote: Total 27695 (delta 147), reused 194 (delta 90), pack-reused 27403
Receiving objects: 100% (27695/27695), 19.89 MiB | 10.56 MiB/s, done.
Resolving deltas: 100% (20453/20453), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/build/LightGBM/python-package/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects

In [22]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

train = pd.read_parquet('/content/gdrive/MyDrive/data/train_after.parquet')

y = train['target']
x = train.drop(columns=['target'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
d_train = lgb.Dataset(x_train, label=y_train,
                      categorical_feature=['road_in_use', 'road_rating',
                                           'road_name', 'connect_code', 'road_type',
                                           'start_node_name',
                                           'start_turn_restricted', 'end_node_name', 'end_turn_restricted'])

params = {}
params['objective'] = 'regression'
params["verbose"] = -1
params['metric'] = 'mae'
params['device_type'] = 'gpu'
params['learning_rate'] = 0.001
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 20
params['min_data'] = 50
params['max_depth'] = 10

bst = lgb.train(params, d_train, 100)

y_pred = bst.predict(x_test)

accuracy = mean_absolute_error(y_pred, y_test)
print(accuracy)

12.300729097635136


In [7]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 41.4 MB/s 
     |████████████████████████████████| 209 kB 69.1 MB/s 
     |████████████████████████████████| 81 kB 9.3 MB/s 
     |████████████████████████████████| 78 kB 8.6 MB/s 
     |████████████████████████████████| 112 kB 75.2 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
     |████████████████████████████████| 147 kB 76.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ed5f3e38b1aa707dbb2d560b9acb4578cf843b7536e9a3e93e01596e8a771b8c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.0.0
    Uninstalling importlib-metadata-5.0.0:
      Successfully uninstalled importlib-metadata-5.0.0


In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

sampler = TPESampler(seed=10)

def objective(trial):
    train = pd.read_parquet('/content/gdrive/MyDrive/data/train_after.parquet')
    y = train['target']
    x = train.drop(columns=['target'])
    x_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    d_train = lgb.Dataset(x_train, label=y_train,
                          categorical_feature=['road_in_use', 'road_rating',
                                           'road_name', 'connect_code', 'road_type',
                                           'start_node_name',
                                           'start_turn_restricted', 'end_node_name', 'end_turn_restricted'])
    params = {
        'objective': 'regression',
        "verbose": -1,
        'metric': 'mse',
        'device_type': 'gpu',
        'learning_rate': trial.suggest_float("learning_rate", 1e-8, 1e-2),
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
    }

    # Generate model
    bst = lgb.train(params, d_train, 100)
    MSE = mean_absolute_error(y_val, bst.predict(X_val))
    return MSE


if __name__ == "__main__":
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-10-12 00:57:18,284] A new study created in memory with name: no-name-4ce08e2e-2730-4851-8871-61ead4a511c1
/content/LightGBM/build/LightGBM/python-package/lightgbm/engine.py:143: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2022-10-12 01:02:54,772] Trial 0 finished with value: 4.255941852481473 and parameters: {'learning_rate': 0.007713208719461027, 'num_leaves': 23, 'max_depth': 11, 'min_child_samples': 76, 'n_estimators': 1546, 'subsample': 0.5348779873185086}. Best is trial 0 with value: 4.255941852481473.
/content/LightGBM/build/LightGBM/python-package/lightgbm/engine.py:143: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
